# MDE

Минимальный ожидаемый эффект — это наименьший истинный эффект полученный от изменений, который с уверенностью сможет обнаружить статистический критерий.
Если прокраса в метрике нет, то это не означает отсутствие эффекта. Эффект может и есть, но он точно не больше, чем MDE для α, β и дисперсии.

Допустим, мы запустили A/B. Было накоплено по ~500 наблюдений в каждой ветке эксперимента. Мы хотим проверить метрику с непрерывным распределением и узнать какой “точности” мы добились за это время.

In [1]:
import numpy as np
import random
from scipy.stats import norm
from statsmodels.stats.power import tt_ind_solve_power 

In [2]:
random.seed(32)

N=500
random_integer_array = np.random.normal(0, 1, N)

In [3]:
mu = random_integer_array.mean()
print(f"mu - {mu}")
std = random_integer_array.std()
print(f"std - {std}")
sample_size = len(random_integer_array)
print(f"sample_size - {sample_size}")
alpha = 0.05
power = 0.8

mu - -0.07005942291291374
std - 1.033142980104643
sample_size - 500


### Вариант 1

In [4]:
def estimate_effect_size(sd, n, alpha=alpha, power=power):
    """
    Расчет MDE для баланса 50/50
    :param sd: ско одной группы
    :param n: размер выборки в одной группе
    :return: MDE
    """
    S = np.sqrt((sd**2 / n) + (sd**2 / n))
    M = norm.ppf(q=1-alpha/2) + norm.ppf(q=power)
    return M * S

In [5]:
mde = estimate_effect_size(std, sample_size)
print(f'MDE - {mde}')

MDE - 0.1830603389269891


Мы получили в качестве результата 0.18. Это означает, что эффекты ниже этого значения будут обладать меньшей мощностью. Соответственно, вероятность того, что эффект действительно есть будет меньше. Эффекты выше этого значения, наоборот, будут мощнее.

Теперь сделаем обратную операцию и рассчитаем количество наблюдений, зная Effect Size. При тех же значениях alpha и power, получим тот же самый размер выборки, что был ранее в качестве входного параметра:

In [6]:
def estimate_sample_size(effect_size, sd, alpha=alpha, power=power):
    """
    Расчет N для баланса 50/50
    :param sd: ско одной группы
    :param effect_size: ожидания по изменения в метрику
    :return: N
    """
    M = (norm.ppf(q=1-alpha/2) + norm.ppf(q=power))**2
    return 2 * M * sd**2 / effect_size**2 

In [7]:
effect_size = estimate_effect_size(std, sample_size)
print(f"Samlpe Size - {estimate_sample_size(effect_size, std)}")

Samlpe Size - 500.0000000000001


Ранее было сказано, что эффекты ниже 0.18 будут обладать меньшей мощностью при 500 наблюдениях. Следовательно, больше наблюдений → больше мощность для того же эффекта и выше.

### Вариант 2

In [8]:
print(f"""
Samlpe Size - {tt_ind_solve_power(effect_size=effect_size, alpha=alpha, power=power, ratio=1)}
MDE -  {tt_ind_solve_power(nobs1=sample_size, alpha=alpha, power=power, ratio=1)}
""")


Samlpe Size - 469.3959498776776
MDE -  0.17735842307242328



### Вариант 3

In [9]:
def get_minimal_determinable_effect(std, sample_size, alpha=alpha, beta=(1-power)):
    t_alpha = norm.ppf(1 - alpha / 2, loc=0, scale=1)
    t_beta = norm.ppf(1 - beta, loc=0, scale=1)
    disp_sum_sqrt = (2 * (std ** 2)) ** 0.5
    mde = (t_alpha + t_beta) * disp_sum_sqrt / np.sqrt(sample_size)
    return mde

In [10]:
mde = get_minimal_determinable_effect(std, sample_size)
print(f'MDE - {mde}')

MDE - 0.18306033892698914
